In [ ]:
# !pip install praw

In [43]:
import requests
import requests.auth
import pandas as pd
from newspaper import Article 
import praw
import json

In [44]:
with open('config.json') as f:
    data = json.load(f)

In [46]:
reddit = praw.Reddit(**data)

In [47]:
print(reddit.user.me())

jaisal1024


In [42]:
list_ = []
for submission in reddit.subreddit('news').controversial(time_filter = 'week', limit=25):
    dict_ = {
            "Title" : submission.title,
            "url" : submission.url, 
            "permalink" : submission.permalink, 
            "Comments" : submission.comments
        }
    list_.append(dict_)
df = pd.DataFrame(list_)
df

,Comments,Title,permalink,url
0,"(ealsmk9, ealn2m7, ealpyzz, ealvovd, eam4lof, ...",Mexico asks U.S. to investigate use of tear ga...,/r/news/comments/a0yqgr/mexico_asks_us_to_inve...,https://www.reuters.com/article/us-usa-immigra...
1,"(ealn2qt, ealslr5, ealqybu, ealopo5, ealogha, ...",California bar shooter had enough ammo to kill...,/r/news/comments/a0zn0a/california_bar_shooter...,https://apnews.com/412a730295ce4b959bb1fc9b08a...
2,"(eak84ri, eak873h, eak8an9, eak8s9d, eak9xup, ...",Alibaba founder Jack Ma is a member of the Com...,/r/news/comments/a0tn6n/alibaba_founder_jack_m...,https://www.cnn.com/2018/11/27/business/jack-m...
3,"(ean7k8f, ean8n2e, ean7247, eanac9n, eapvqy5)",Restaurant chain bans mobile phones.,/r/news/comments/a170xc/restaurant_chain_bans_...,https://www.bbc.co.uk/news/amp/business-46369736
4,"(eav9y2c, eauflrq, easviiq, eat0wj4)","Menial Tasks, Slurs and Swastikas: Many Black ...",/r/news/comments/a1uvzj/menial_tasks_slurs_and...,https://www.nytimes.com/2018/11/30/business/te...
5,"(eaon3jl, eaoo0u8, eaozqwz, eaoz28r, eaonagi, ...",Chance US missionary ‘shot dead’ on North Sent...,/r/news/comments/a1cnol/chance_us_missionary_s...,https://www.news.com.au/travel/travel-updates/...
6,"(eas5q18, eas9f76, easixgh, easll1l, easbl66, ...",Christian Artists Could Face Jail in AZ for Re...,/r/news/comments/a1rkjm/christian_artists_coul...,http://insider.foxnews.com/2018/11/29/christia...
7,"(eap1u03, eaq2gui, eap5fbp, eapt8ar, eaq4lmd, ...",No end set for immigrant kids' internment at '...,/r/news/comments/a1e763/no_end_set_for_immigra...,http://www.msnbc.com/rachel-maddow/watch/no-en...
8,"(eany97c, eanzcgu, eaod7hn, eaqy2vp, eaohzlu, ...",Man who attacked Muslim woman in hospital was ...,/r/news/comments/a19q2q/man_who_attacked_musli...,https://m.metrotimes.com/news-hits/archives/20...
9,"(eawf5w3, eawwbmk, eawgten, eaxa7lt, eaykd15, ...",Las Vegas man blames flu shot for triggering r...,/r/news/comments/a2a464/las_vegas_man_blames_f...,https://news3lv.com/news/local/las-vegas-man-b...
